In [1]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from xgboost import XGBRegressor
import math
from sklearn.metrics import mean_squared_error
import dask.dataframe as dd
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from dask_ml.model_selection import RandomizedSearchCV
from tqdm import tqdm
import numpy as np
import dask_ml.model_selection as dcv
from sklearn.linear_model import Lasso,Ridge

from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
import pickle
from joblib import dump, load
import math

In [8]:
! pip install dask

In [4]:
print(7%6)

1


In [9]:
!pip install xgboost

In [25]:
!pip install dask[dataframe]

# Preprocessing Data

In [2]:
#path = r'/content/drive/My Drive/case study1/train_final_with_outlier.csv'
train_final = pd.read_csv('train_final_with_outlier.csv')
train_final.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,Electricity,2016-01-01 00:00:00,0.0,0,Education,8,2008.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,Jan,Friday
1,1,Electricity,2016-01-01 00:00:00,0.0,0,Education,160,2004.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,Jan,Friday
2,2,Electricity,2016-01-01 00:00:00,0.0,0,Education,0,1991.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,Jan,Friday
3,3,Electricity,2016-01-01 00:00:00,0.0,0,Education,133,2002.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,Jan,Friday
4,4,Electricity,2016-01-01 00:00:00,0.0,0,Education,127,1975.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,Jan,Friday


In [3]:
test_final = pd.read_csv('test_final.csv')
test_final.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,0,0,2017-01-01 00:00:00,0,Education,8,2008.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
1,1,1,0,2017-01-01 00:00:00,0,Education,160,2004.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
2,2,2,0,2017-01-01 00:00:00,0,Education,0,1991.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
3,3,3,0,2017-01-01 00:00:00,0,Education,133,2002.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
4,4,4,0,2017-01-01 00:00:00,0,Education,127,1975.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6


In [4]:
train_final.shape,test_final.shape

((20216100, 20), (41697600, 20))

In [5]:
y = train_final['meter_reading']
train_final.drop('meter_reading',axis=1,inplace=True)

In [6]:
test_final.drop('row_id',axis=1,inplace=True)

In [7]:
train_final.shape,test_final.shape

((20216100, 19), (41697600, 19))

In [8]:
train_final['month'].unique()

array(['Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'Aug',
       'Sept', 'Oct', 'Nov', 'Dec'], dtype=object)

In [9]:
test_final['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [10]:
''' replace the month name with the month type'''
month={'Jan':1,'Feb':2,'March':3,'April':4,'May':5,'June':6,'July':7,'Aug':8,'Sept':9,'Oct':10,'Nov':11,'Dec':12}
train_final['month']=train_final['month'].map(month)
train_final.head()


,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,Electricity,2016-01-01 00:00:00,0,Education,8,2008.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
1,1,Electricity,2016-01-01 00:00:00,0,Education,160,2004.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
2,2,Electricity,2016-01-01 00:00:00,0,Education,0,1991.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
3,3,Electricity,2016-01-01 00:00:00,0,Education,133,2002.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
4,4,Electricity,2016-01-01 00:00:00,0,Education,127,1975.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday


In [11]:
train_final['month'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

# we split the data based on the month as we have chosen all the data upto October as train data, and the data of November and December as test data

In [12]:
x_train=train_final.iloc[0:16800257]
x_cv=train_final.iloc[16800257:]
y_train=y.iloc[0:16800257]
y_cv=y.iloc[16800257:]

In [13]:
x_train.shape,x_cv.shape

((16800257, 19), (3415843, 19))

In [14]:
y_train.shape,y_cv.shape

((16800257,), (3415843,))

In [16]:
''' Preprocrssing of 'primary_use' data'''
le = LabelEncoder()
le.fit(x_train['primary_use'])

x_train['primary_use'] = le.transform(x_train['primary_use'])
x_cv['primary_use'] = le.transform(x_cv['primary_use'])
test_final['primary_use'] = le.transform(test_final['primary_use'])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
x_train.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,Electricity,2016-01-01 00:00:00,0,0,8,2008.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
1,1,Electricity,2016-01-01 00:00:00,0,0,160,2004.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
2,2,Electricity,2016-01-01 00:00:00,0,0,0,1991.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
3,3,Electricity,2016-01-01 00:00:00,0,0,133,2002.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday
4,4,Electricity,2016-01-01 00:00:00,0,0,127,1975.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,Friday


In [22]:
''' Preprocrssing of 'weekday' data'''
le = LabelEncoder()
le.fit(x_train['weekday'])

x_train['weekday'] = le.transform(x_train['weekday'])
x_cv['weekday'] = le.transform(x_cv['weekday'])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
x_train['weekday'].unique()

array([0, 2, 3, 1, 5, 6, 4])

In [24]:
''' Preprocrssing of 'meter' data'''
le = LabelEncoder()
le.fit(x_train['meter'])

x_train['meter'] = le.transform(x_train['meter'])
x_cv['meter'] = le.transform(x_cv['meter'])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
x_train.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,0,2016-01-01 00:00:00,0,0,8,2008.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
1,1,0,2016-01-01 00:00:00,0,0,160,2004.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
2,2,0,2016-01-01 00:00:00,0,0,0,1991.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
3,3,0,2016-01-01 00:00:00,0,0,133,2002.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
4,4,0,2016-01-01 00:00:00,0,0,127,1975.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0


In [26]:
x_train['meter'].unique()

array([0, 1, 3, 2])

In [27]:
''' to save the x_train csv file for future use'''
#path = r'/content/drive/My Drive/case study1/x_train.csv'
x_train.to_csv('x_train.csv',index=False)

In [28]:
''' to save the x_cv csv file for future use'''
#path = r'/content/drive/My Drive/case study1/x_cv.csv'
x_cv.to_csv('x_cv.csv',index=False)

In [22]:
''' to save the x_train csv file for future use'''
#path = r'/content/drive/My Drive/case study1/x_train.csv'
y_train.to_csv('y_train.csv',index=False)

In [69]:
''' to save the x_cv csv file for future use'''
#path = r'/content/drive/My Drive/case study1/y_cv.csv'
y_cv.to_csv('y_cv.csv',index=False)

In [24]:
''' to save the x_test csv file for future use'''
test_final.to_csv('x_test.csv',index=False)

# Model 

In [2]:
x_train=pd.read_csv('x_train.csv')
x_train.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,0,2016-01-01 00:00:00,0,0,8,2008.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
1,1,0,2016-01-01 00:00:00,0,0,160,2004.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
2,2,0,2016-01-01 00:00:00,0,0,0,1991.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
3,3,0,2016-01-01 00:00:00,0,0,133,2002.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0
4,4,0,2016-01-01 00:00:00,0,0,127,1975.0,3.0,25.0,6.0,20.0,0.796416,1019.7,0.0,0.0,0,1,1,0


In [3]:
x_cv=pd.read_csv('x_cv.csv')
x_cv.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,0,2016-11-01 00:00:00,0,0,8,2008.0,3.0,23.3,2.0,18.3,0.0,1018.1,60.0,3.6,0,1,11,5
1,1,0,2016-11-01 00:00:00,0,0,160,2004.0,3.0,23.3,2.0,18.3,0.0,1018.1,60.0,3.6,0,1,11,5
2,2,0,2016-11-01 00:00:00,0,0,0,1991.0,3.0,23.3,2.0,18.3,0.0,1018.1,60.0,3.6,0,1,11,5
3,3,0,2016-11-01 00:00:00,0,0,133,2002.0,3.0,23.3,2.0,18.3,0.0,1018.1,60.0,3.6,0,1,11,5
4,4,0,2016-11-01 00:00:00,0,0,127,1975.0,3.0,23.3,2.0,18.3,0.0,1018.1,60.0,3.6,0,1,11,5


In [2]:
x_test=pd.read_csv('x_test.csv')
x_test.head()

,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,month,weekday
0,0,0,2017-01-01 00:00:00,0,0,8,2008.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
1,1,0,2017-01-01 00:00:00,0,0,160,2004.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
2,2,0,2017-01-01 00:00:00,0,0,0,1991.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
3,3,0,2017-01-01 00:00:00,0,0,133,2002.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6
4,4,0,2017-01-01 00:00:00,0,0,127,1975.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6,0,1,1,6


In [4]:
y_cv=pd.read_csv('y_cv.csv')
y_train=pd.read_csv('y_train.csv')

# Replace None value of test data

In [11]:
x_test['year_built'].fillna(x_test['year_built'].median(),inplace  = True)
x_test['floor_count'].fillna(x_test['floor_count'].median(),inplace  = True)
x_test['air_temperature'].fillna(x_test['air_temperature'].mean(),inplace  = True)
x_test['cloud_coverage'].fillna(x_test['cloud_coverage'].mean(),inplace  = True)
x_test['dew_temperature'].fillna(x_test['dew_temperature'].mean(),inplace  = True)
x_test['precip_depth_1_hr'].fillna(x_test['precip_depth_1_hr'].mean(),inplace  = True)
x_test['sea_level_pressure'].fillna(x_test['sea_level_pressure'].mean(),inplace  = True)
x_test['wind_direction'].fillna(x_test['wind_direction'].mean(),inplace  = True)
x_test['wind_speed'].fillna(x_test['wind_speed'].mean(),inplace  = True)

In [14]:
''' to save the x_test csv file for future use'''
x_test.to_csv('x_test_preprocessed.csv',index=False)

In [5]:
x_train.drop('timestamp',axis=1,inplace=True)
x_cv.drop('timestamp',axis=1,inplace=True)

In [3]:
x_test.drop('timestamp',axis=1,inplace=True)

In [7]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype
def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [8]:
x_train = reduce_mem_usage(x_train,use_float16=True)

Memory usage of dataframe is 2307.16 MB
Memory usage after optimization is: 464.64 MB
Decreased by 79.9%


In [7]:
x_train.shape

(16800257, 18)

In [8]:
x_cv.shape

(3415843, 18)

In [13]:
x_test.shape

(41697600, 18)

# Custom Ensemble Model Implementation(Stacking)

# Create dataset

1. Split the total data into train and cv (80-20)

2. Split x_train into D1 & D2 (50-50)

3. From D1 creat 10 dataset doing sampling with replacement


In [12]:
D1=x_train.iloc[0:8208933]
D2=x_train.iloc[8208933:]
y_D1=y_train.iloc[0:8208933]
y_D2=y_train.iloc[8208933:]

In [13]:
D1.to_csv('D1.csv',index=False)
D2.to_csv('D2.csv',index=False)
y_D1.to_csv('y_D1.csv',index=False)
y_D2.to_csv('y_D2.csv',index=False)

In [21]:
D1.shape,D2.shape

((8208933, 18), (8591324, 18))

# Final Metemodel

# Custom model of Stacking

In [10]:
D1 = pd.read_csv('D1.csv')
y_D1 = pd.read_csv('y_D1.csv')
D2 = pd.read_csv('D2.csv')
y_D2 = pd.read_csv('y_D2.csv')

In [4]:
D1.shape

(8208933, 18)

In [37]:
EnsembleModelDict = {1:Lasso(),2:Ridge(),3:ElasticNet(),4:XGBRegressor(),5:DecisionTreeRegressor(),0:LinearRegression()}
model_name={1:'Lasso',2:'Ridge',3:'ElasticNet',4:'XGBRegressor',5:'DecisionTreeRegressor',0:'LinearRegression'}
hyper_param={1:{'alpha':[0.0001, 0.001, 0.01, 0.1,10]},2:{'alpha':[0.0001, 0.001, 0.01, 0.1,10]},\
            3:{'alpha':[0.0001, 0.001, 0.01, 0.1,10],"l1_ratio": np.arange(0.0, 1.0, 0.1)},\
            4:{'learning_rate':  [0.0001, 0.001, 0.01, 0.1,.2,.3] , 'n_estimators': [5,10,50, 75,100,150],\
               'gamma':[0.5, 1, 1.5, 2,2.5,5],'max_depth':[3, 4, 5]},\
            5:{"max_depth": [2, 6, 8],"min_samples_leaf": [20, 40, 100],"max_leaf_nodes": [5, 20, 100]}}
i=1
df = pd.DataFrame()
df_test = pd.DataFrame()
Stackmodellist=list()
while (i<=10):
    selecting_row=np.random.choice(D1.shape[0],int((40*D1.shape[0])/100),replace=True)
    sample_data=np.take(D1,selecting_row,axis=0)
    target_of_sample_data=np.take(y_D1,selecting_row,axis=0)
    #print(sample_data.shape,target_of_sample_data.shape)
    k=i%len(EnsembleModelDict)
    if hyper_param.get(k)!=None:                                               

        clf_hyperparam = RandomizedSearchCV(EnsembleModelDict[k], hyper_param[k], n_iter=30,cv=5,return_train_score=True,n_jobs=-1)
        clf_hyperparam.fit(sample_data, target_of_sample_data)

        best_params = clf_hyperparam.best_params_
        target_of_sample_data_pred_Model_D2,Model_pkl=ModelPrediction(model_name[k],best_params,sample_data,target_of_sample_data,D2,i)
        Model_pkl_cv = pickle.loads(Model_pkl)
        Stackmodellist.append(Model_pkl_cv)
        target_of_sample_data_pred_Model_x_cv=ModelPrediction_cv(Model_pkl_cv,x_cv)
    else:
        best_params=None
        target_of_sample_data_pred_Model_D2,Model_pkl=ModelPrediction(model_name[k],best_params,sample_data,target_of_sample_data,D2,i)
        Model_pkl_cv = pickle.loads(Model_pkl)
        Stackmodellist.append(Model_pkl_cv)
        target_of_sample_data_pred_Model_x_cv=ModelPrediction_cv(Model_pkl_cv,x_cv)
   
    df = pd.concat([df,pd.DataFrame(target_of_sample_data_pred_Model_D2)],axis=1)
    df.to_csv('CustomEnsembleModels_Custom_Stacking.csv',index=False)
    df_test = pd.concat([df_test,pd.DataFrame(target_of_sample_data_pred_Model_x_cv)],axis=1)
    df_test.to_csv('CustomEnsembleModels_Custom_Stacking_cv.csv',index=False)
    print('Model:{0} is completed'.format(model_name[k]))
    
    i+=1

(3283573, 18) (3283573, 1)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:289: UserWarning: The total space of parameters 5 is smaller than n_iter=30. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Model:Lasso is completed
(3283573, 18) (3283573, 1)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:289: UserWarning: The total space of parameters 5 is smaller than n_iter=30. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Model:Ridge is completed
(3283573, 18) (3283573, 1)


...

Model:Lasso is completed
(3283573, 18) (3283573, 1)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:289: UserWarning: The total space of parameters 5 is smaller than n_iter=30. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Model:Ridge is completed
(3283573, 18) (3283573, 1)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5881895.324738828, tolerance: 1296.4912068208082
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5887427.928867648, tolerance: 1297.3548290967667
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5887403.965210735, tolerance: 1297.7169750537516
  positive)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5886793.1

Model:ElasticNet is completed
(3283573, 18) (3283573, 1)
Model:XGBRegressor is completed


In [9]:
def ModelPrediction(modelname,best_params,sample_data,target_of_sample_data,Test_Data,iteration):
    if modelname == 'Ridge':
        Model = Ridge( alpha= best_params['alpha'])
    elif modelname == 'Lasso':
        Model = Lasso( alpha= best_params['alpha'])
    elif modelname == 'ElasticNet':
        Model = ElasticNet(alpha= best_params['alpha'],l1_ratio= best_params['l1_ratio'])
    elif modelname == 'XGBRegressor':  
        Model = XGBRegressor( booster='gbtree', gamma=best_params['gamma'],learning_rate=best_params['learning_rate'],\
                             max_depth=best_params['max_depth'], min_child_weight=1,n_estimators=best_params['n_estimators'],\
                             n_jobs=1, random_state=0)
    
    elif modelname == 'DecisionTreeRegressor':
        Model = DecisionTreeRegressor( criterion='mse', max_depth=best_params['max_depth'], min_samples_split=2,\
                min_samples_leaf=best_params['min_samples_leaf'],max_leaf_nodes=best_params['max_leaf_nodes'])
    elif modelname == 'LinearRegression':
        Model = LinearRegression(n_jobs=-1)

    Model.fit(sample_data, target_of_sample_data)
    target_of_sample_data_pred_Model = Model.predict(Test_Data)
    Model_pkl = pickle.dumps(Model)
    filename= modelname+'_'+str(iteration)+'.joblib'
    dump(Model, filename)
    print('{0} - model saved with filename as {1}'.format(modelname,filename))
    return target_of_sample_data_pred_Model,Model_pkl

In [36]:
def ModelPrediction_cv(Model,Test_Data):
    target_of_cv_data_pred_Model = Model.predict(Test_Data)
    return target_of_cv_data_pred_Model

In [12]:
df_D2=pd.read_csv('CustomEnsembleModels_Custom_Stacking.csv')
df_D2.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,2.795826,2.802002,2.803146,-0.294550,0.092344,2.805014,2.797141,2.809782,2.799896,0.235334
1,2.779713,2.789634,2.791012,-0.044361,0.092344,2.788877,2.782127,2.794041,2.786762,-0.002473
2,2.773371,2.781072,2.780179,0.174686,0.092344,2.783412,2.775372,2.786768,2.778096,0.051948
3,2.782435,2.791947,2.792962,-0.132729,0.092344,2.791683,2.784763,2.796555,2.789040,0.308179
4,2.742111,2.754212,2.752026,0.412863,0.092344,2.752770,2.745580,2.755433,2.749932,0.138090


In [14]:
df_x_cv=pd.read_csv('CustomEnsembleModels_Custom_Stacking_cv.csv')
df_x_cv.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,3.090364,3.092554,3.075002,-0.210833,0.092344,3.086222,3.075489,3.094225,3.075063,0.173885
1,3.074251,3.080185,3.062868,0.039356,0.092344,3.070086,3.060475,3.078483,3.061929,-0.012178
2,3.067909,3.071624,3.052035,0.187588,0.092344,3.064620,3.053721,3.071210,3.053264,0.008297
3,3.076973,3.082498,3.064818,-0.049012,0.092344,3.072891,3.063111,3.080997,3.064207,0.341465
4,3.036650,3.044763,3.023882,0.471545,0.092344,3.033979,3.023928,3.039876,3.025099,0.141878


# prediction of x_cv

In [6]:
from xgboost import XGBRegressor
import math
neigh = XGBRegressor()
parameters ={'learning_rate ':  [0.0001, 0.001, 0.01, 0.1,.2,.3] , 'n_estimators': [5,10,50, 75,100,150],'gamma':[0.5, 1, 1.5, 2,2.5,5],'max_depth':[3, 4, 5]}
clf = RandomizedSearchCV(neigh, parameters, n_iter=30,cv=5,return_train_score=True,n_jobs=-1)
clf.fit(df_D2, y_D2)

print(clf.best_params_)

{'n_estimators': 100, 'max_depth': 5, 'learning_rate ': 0.001, 'gamma': 5}


In [15]:
model = XGBRegressor( booster='gbtree', gamma=5,learning_rate=0.001,\
                             max_depth=5, min_child_weight=1,n_estimators=100,\
                             n_jobs=1, random_state=0)
model.fit(df_D2, y_D2)

y_D2_pred = model.predict(df_D2)
y_cv_pred = model.predict(df_x_cv)

MSE_TRAIN=mean_squared_error(y_D2, y_D2_pred)
MSE_TEST=mean_squared_error(y_cv, y_cv_pred)
RMSLE_TRAIN=np.sqrt(mean_squared_log_error(y_D2, y_D2_pred))
RMSLE_TEST=np.sqrt(mean_squared_log_error(y_cv, y_cv_pred))

print(RMSLE_TRAIN,RMSLE_TEST)

1.0619798297667795 1.0446521746675752


# Final prediction

# Test data

In [2]:
x_test_preprocessed=pd.read_csv('x_test_preprocessed.csv')

In [3]:
def batch_test(test_data,start_index,end_index):
    test_batch_data = test_data[start_index:end_index]
    return test_batch_data

In [4]:
model_name={1:'Lasso',2:'Ridge',3:'ElasticNet',4:'XGBRegressor',5:'DecisionTreeRegressor',0:'LinearRegression'}
df = pd.DataFrame()
i=1
step_size = math.floor(x_test_preprocessed.shape[0]/20)
while(i<=10):
    k=i%6
    test_df = pd.DataFrame()
    modelname=model_name[k]+'_'+str(i)+'.joblib'
    Model = load(modelname)
    for start_pos in range(0,x_test_preprocessed.shape[0],step_size):
        start_index,end_index = start_pos,x_test_preprocessed.shape[0] if start_pos+step_size > x_test_preprocessed.shape[0] else start_pos+step_size
        target_of_test_data_pred_Model = Model.predict(batch_test(x_test_preprocessed,start_index,end_index))
        test_df=pd.concat([test_df,pd.DataFrame(target_of_test_data_pred_Model)],axis=0)
    df = pd.concat([df,test_df],axis=1)
    df.to_csv('CustomEnsembleModels_Custom_Stacking_test_data.csv',index=False)
    print('model-{0} is completed with Test Data'.format(model_name[k]))
    i+=1

model-Lasso is completed with Test Data
model-Ridge is completed with Test Data
model-ElasticNet is completed with Test Data
model-XGBRegressor is completed with Test Data
model-DecisionTreeRegressor is completed with Test Data
model-LinearRegression is completed with Test Data
model-Lasso is completed with Test Data
model-Ridge is completed with Test Data
model-ElasticNet is completed with Test Data
model-XGBRegressor is completed with Test Data


In [7]:
x_test_CustomEnsembleModels_pred=pd.read_csv('CustomEnsembleModels_Custom_Stacking_test_data.csv')
x_test_CustomEnsembleModels_pred.shape

(41697600, 10)

In [8]:
x_test_CustomEnsembleModels_pred.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,2.559588,2.562818,2.561123,-0.362294,0.032081,2.558432,2.565374,2.570279,2.561369,0.099504
1,2.543475,2.550449,2.548989,-0.186750,0.032081,2.542296,2.550359,2.554538,2.548236,-0.107265
2,2.537133,2.541887,2.538156,0.128873,0.032081,2.536830,2.543605,2.547265,2.539570,0.102945
3,2.546197,2.552762,2.550940,-0.280069,0.032081,2.545101,2.552996,2.557052,2.550514,0.118932
4,2.505873,2.515027,2.510003,0.367050,0.032081,2.506188,2.513813,2.515930,2.511406,-0.005599


In [13]:
model = XGBRegressor( booster='gbtree', gamma=5,learning_rate=0.001,\
                             max_depth=5, min_child_weight=1,n_estimators=100,\
                             n_jobs=1, random_state=0)
model.fit(df_D2, y_D2)

y_test_pred = model.predict(x_test_CustomEnsembleModels_pred)


In [18]:
type(y_test_pred)

numpy.ndarray

In [21]:
pred=pd.DataFrame(y_test_pred,columns=['pred'])

In [28]:
pred.describe()

,pred
count,4.169760e+07
mean,6.082368e-01
std,2.719319e-01
min,4.539930e-01
25%,7.420859e-01
50%,8.448242e-01
75%,9.609123e-01
max,1.229294e+00
